In [1]:
## Importación al bucket iedatalakeprocesss de información extraida del CRM y Solicitud Online 
## -------------------------------------------------------------------------------------------
## Información unificada y transformada para TC LAB (Lab para The Cocktail)

import sqlalchemy
import csv
import string
import pandas as pd
import os
import shutil
import datetime
import uuid
import sys

# Importación de opciones de configuración

if (os.name == 'nt'):
    sys.path.append(os.environ["USERPROFILE"].replace('\\', '/') + '/iedatalake/python/config')
else: 
    sys.path.append('/home/ubuntu/iedatalake/python/config')
import databaseconfig as dbc


In [2]:
## Proceso, versíón y fecha de la transformación para identificar la ejecución

transformationProcess = 'Transformacion de CRM y Solicitud Online desde analytics01 en python'
transformationVersion = str(uuid.uuid1())
transformationInitialDate = str(datetime.datetime.now())

## Carpeta auxiliar para depositar los CSVs generados antes de subir

if not os.path.exists("data"):
    os.makedirs("data")


In [3]:
## Procedimiento de extracción de datos de CRM a un fichero CSV local

def ExtractFromCRM(tableName, **optionalParameters): 
    
    ## Motor SQLAlchemy para la base de datos SQL Server réplica del CRM online 

    crmEngine = sqlalchemy.create_engine(dbc.crm['dialect'] + '+' + dbc.crm['driver'] + '://' \
      + dbc.crm['username'] + ':' + dbc.crm['password'] + '@' \
      + dbc.crm['host'] + '/' + dbc.crm['database'] + dbc.crm['parameters'])
    
    # Ruta del fichero CSV, asumida en una subcarpeta "data" en la carpeta donde se ejecuta
    # y extensión .csv.gz porque se generará comprimido en formato GZIP
    
    csvFile = optionalParameters.get('csvFile', 'data/CRM.' + tableName + '.csv.gz')
    
    # Parámetro opcional para indicar si requiere un procedimiento previo auxiliar "Aux"
    
    auxProcedure = optionalParameters.get('auxProcedure', None)
    
    # Parámetro opcional con el índice a aplicar al fichero CSV
    
    csvIndex = optionalParameters.get('csvIndex', None)
    if (csvIndex == None):
        csvIndex = tableName + 'Id'        
        
    # Recuperación por base de datos de tabla resultante de la transformación
    
    crmConnection = crmEngine.connect()
    sqlSentence = ''
    if (auxProcedure != None):
        sqlSentence += 'EXEC IE_Vistas.BI.' + auxProcedure + '\n'
    sqlSentence += "EXEC IE_Vistas.BI." + tableName
    if (auxProcedure != None):
        sqlSentence += '\nDROP TABLE Aux' + tableName + '\n'
    table = pd.io.sql.read_sql(sqlSentence, crmConnection)
    table.set_index(csvIndex, inplace = True)
    table.to_csv(csvFile, sep = ';', compression = 'gzip', quotechar='|', quoting=csv.QUOTE_MINIMAL, 
                 encoding = 'utf8')
    crmConnection.close()
      
        

In [4]:
## CRM.Person
## (PersonId, Fullname, Gender, EmailAddress, Telephone, ContactType, CandidateStatus, InterestType, 
## InitialContactCategory, InitialContact, PrimaryNationality, HomeCountry, MktCountry, MktGeographicalArea, 
## MktProvince, MktCity, TotalYearsExperience, SubjectiveClassification, CommunicationLanguage, VisitedCountries, 
## IEFit, LastUpdate, CreationDate)

## Personas activas en CRM y atributos de interés (nombre completo, género, e-mail, teléfono, tipo de contacto, 
## estado de contacto, tipo de interés, contacto inicial y su categoría, nacionalidad primaria, país de 
## procedencia, atributos de la dirección de Marketing, idioma de comunicación, clasificación subjetiva, 
## si encaja en IE, fecha de creación). Hay dos campos calculados:  

## - TotalYearsOfExperience: Estima el total de años de experiencia de una persona a partir de sus experiencias
##   profesionales. No era viable en los térmicos solicitados sumando los años de experiencia en los distintos
##   cargos, al haber muchos casos con experiencias solapadas en el tiempo y cuya suma de años produce una 
##   cantidad enorme y sin el sentido deseado. En su lugar, se han calculado los segmentos sin solapamiento 
##   de los intervalos de tiempo entre las fechas de inicio y fin de las experiencias profesionales, y sobre 
##   ellos estimado el total de años de experiencia. Al ser un cálculo iterativo que va recorriendo cada 
##   experiencia de cada persona para determinar las intersecciones y acumular sólo segmentos no solapados, 
##   emplea unos 15 minutos en completarse para todos los contactos y experiencias activas.

## - VisitedCountries: Estima el número de países visitados por el contacto, a partir de los países que aparecen
##   en sus formaciones y experiencias profesionales, sumando la cantidad de países distintos que aparecen y
##   restándole uno, asumiendo que uno de ellos debe ser su país de origen, al no servir para esto el país de 
##   origen del contacto por venir sin informar en muchos casos, ni el país de la dirección de Marketing al 
##   tener carácter comercial y no tener que coincidir con el de origen. 

ExtractFromCRM('Person')
    

In [5]:
## CRM.Program 
## (ProgramId, Name, ShortName, Category, ProgramType, ProgramGroup, Format, Language, InAcademicCatalog, 
## InCommercialCatalog, Status, Commercialized)

## Maestro de programas, incluyendo todos salvo los ficticios, su clasificación jerárquica Category / 
## ProgramType / ProgramGroup, formato, idioma, pertenencia al catálogo académico, pertenencia al catálogo 
## comercial, su estado activo o inactivo y si es o no comercializable. 

ExtractFromCRM('Program')

In [6]:
## CRM.ProgramSchools 
## (ProgramId, School)

## Los programas no se vinculan a una única escuela del IE, sino que hay una asociación múltiple que 
## permite relacionar un programa con varias escuelas. Por este motivo no se incluye la escuela en el 
## maestro de programas sino en esta tabla aparte con los programas no ficticios y un registro por cada 
## escuela asociada. 

ExtractFromCRM('ProgramSchools')

In [7]:
## CRM.Offering 
## (OfferingId, ProgramId, Intake, Season, AcademicYear, StartingDate, EndingDate, SalesDeadLine, Status, 
## Commercialized)

## Ofertas formativas de los distintos programas. Cada una va definida por la convocatoria (Intake), su programa
## asociado, el año académico (AcademicYear) y estación (Season) calculados a partir del nombre oficial, fecha 
## de inicio, fecha de fin y fecha de cierre de ventas (SalesDeadLine), calculada a partir de la fecha de inicio
## y tipo del programa ofertado, el estado (activo o inactivo) y si es o no comercializable. 

## Aunque muchas aplicaciones, procesos e informes requieren que el nombre oficial de convocatoria (Intake) 
## esté bien formado según el formato MMM-AAAA (por ejemplo, OCT-2016), o bien que sean activas y/o 
## comercializables al igual que sus programas, aquí se incluyen todas, excluyéndose sólo si su programa es
## ficticio. 

ExtractFromCRM('Offering')

In [8]:
## CRM.EventRegistration 
##(EventRegId, PersonId, EventName, EventDate, EventRegDate, EventType, Attendance, GeographicalArea, 
## Country, Province, City, Location)

## Información de registros a eventos: la persona inscrita, los datos del evento (nombre, fecha, tipo, 
## atributos de ubicación) y del registro: fecha de registro (EventRegDate) y estado de la inscripción 
## (Attendance). No ha sido posible incluir la fecha de asistencia al evento pues, aunque existe internamente 
## un atributo para ello, no se está rellenando desde el formulario de CRM.

## Sólo se incluyen registros activos a eventos activos, pero no se limita nada más. Para restringir a 
## registros a eventos con asistencia confirmada, se aplica el filtro Attendance = ‘Confirmed attendance’. 

ExtractFromCRM('EventRegistration', csvIndex = 'EventRegId')

In [9]:
## CRM.Opportunity 
## (OpportunityId, PersonId, ProgramId, OfferingId, EventRegId, OpportunityStatus, StartingDate,
## EndingDate, OnlineApplicationDate, OnlineApplicationProgress, OppInitialContactCategory, OppInitialContact, 
## CreationDate, LastUpdate)

## Oportunidades en CRM recogiendo el interés de una persona en un programa entre una determinada fecha de
## inicio y otra de fin, en un estado de oportunidad concreto. En ocasiones, se relaciona con una oferta 
## formativa concreta, con un registro a evento concreto (calculado relacionando varias entidades de CRM). 
## También informa a veces el canal de entrada (categoría y contacto inicial), la fecha de inicio de la 
## solicitud online y su progreso. Sólo se ha restringido en la carga que la persona sea activa, dando 
## cabida a los distintos casos (sin programa, varias para un mismo contacto y programa, etc.) a fin de 
## reflejar la casuística real que hay.  

## Disponer del canal de entrada en la oportunidad y en la persona hace posible un análisis separado. 
## Por ejemplo, el valor de contacto inicial “Royall” se podría filtrar en la oportunidad en 
## OppInitialContactCategory o en la persona en InitialContatCategory.

ExtractFromCRM('Opportunity', auxProcedure = 'ObtenerAuxOpportunity')

In [10]:
## CRM.Enrollment 
## (EnrollmentId, OfferingId, PersonId, EnrollmentStatus, FeePaid, Recommenders, OpportunityId, 
## LastStatusChange, LastUpdate)

## Inscripciones en las ofertas formativas. Se obtienen de admisiones activas para personas activas, quedando
## relacionadas generalmente con una persona, una oferta formativa y una oportunidad, incluyendo el estado de
## admisión, último cambio de estado, información de tasa pagada, y los recomendantes separados por comas 
## calculados en un solo atributo (Recommenders), en caso de tenerlos.

ExtractFromCRM('Enrollment', auxProcedure = 'ObtenerAuxEnrollment')

In [11]:
## CRM.FinancialAid 
## (FinancialAidId, EnrollmentId, FinancialAidStatus, AmountGranted, Revision, DirectAward, ClauseAccepted, 
## Fellowship1, Fellowship1Type, Fellowship1Status, Fellowship1Amount, Fellowship2, Fellowship2Type, 
## Fellowship2Status, Fellowship2Amount, Fellowship3, Fellowship3Type, Fellowship3Status, Fellowship3Amount, 
## LastStatusChange)

## Ayudas financieras activas asociadas a inscripciones en ofertas formativas para contactos activos junto 
## con sus tres posibles becas activas y tipología o nombre de descuento de éstas, así como el estado de la 
## ayuda y estado de cada una de las becas, y sus importes. Hay un campo adicional AmountGranted donde se ha
## calculado la suma de los importes (del IE) de las becas aprobadas para las ayudas concedidas.

## Aunque en teoría sólo puede haber tres becas activas para una ayuda, hay casos en CRM con más y, para dar
## cabida a sólo tres como nos indicaron considerar, ha habido que escoger para la carga las tres principales, 
## eligiendo primero las aprobadas y del resto las más actuales. Las becas que se quedan fuera a partir de la
## cuarta deben ser errores y prescindibles, se incluyen en el cálculo de AmountGranted, pero parecen no afectar
## porque no debe haber ninguna aprobada en tales condiciones.

ExtractFromCRM('FinancialAid', auxProcedure = 'ObtenerAuxFinancialAid')

In [20]:
## CRM.AdmissionTest 
## (AdmissionTestId, PersonId, ProgramId, TestName, TestGradeType, Score, Multiscore, LiteralGrade, MinValue, 
## MaxValue, Channel, Date, CreationDate)

## Pruebas de admisión activas para personas activas. Recogen la persona evaluada, en ocasiones un programa 
## concreto al que aplica la prueba, nombre del examen, tipo de calificación esperada y calificación registrada
## en tres posibles formatos dependiendo del formato del resultado y tipo de prueba (Score, Multiscore, 
## LiteralGrade), valor mínimo y máximo posible, canal y fecha. 

ExtractFromCRM('AdmissionTest')

In [4]:
## CRM.AcademicTraining 
## (AcademicTrainingId, PersonId, Title, EducationLevel, AcademicDisplicine, College, Institution, 
## AverageGrade, Country, City, StartingDate, EndingDate)

## Formaciones académicas activas para personas activas, incluyendo persona, título, nivel educativo, 
## rama académica, colegio, institución, nota media (texto libre), país, ciudad, fecha de inicio y fin.

ExtractFromCRM('AcademicTraining')

In [14]:
## CRM.LanguageCompetence 
## (Language, PersonId, LevelNumber, Level, LanguageCompetenceId)

## Competencias en idiomas activas para personas activas, incluyendo persona, idioma y nivel numérico 
## y descrito. En caso de aparecer varias veces un mismo idioma para la misma persona, se carga sólo el 
## de mayor nivel numérico. 

ExtractFromCRM('LanguageCompetence')

In [15]:
## CRM.NationalityRecord (Nationality, Personid, DocumentType, Document, NationalityRecordId)

## Nacionalidades activas para personas activas, incluyendo persona, nacionalidad, tipo de documento y 
## documento identificativo asociado. 

ExtractFromCRM('NationalityRecord')

In [16]:
## CRM.ProfessionalExperience

ExtractFromCRM('ProfessionalExperience', csvIndex = 'ProfExperienceId')

In [17]:
## CRM.OpportunityTask
## (TaskId, Status, CreationDate, ActualEnd)

ExtractFromCRM('OpportunityTask', csvIndex='TaskId')

In [18]:
## CRM.OnlineLeadFollowup
## (FollowupId, OpportunityId, Enrollmentid, TaskType, Subject, CallFeedback, Reasons, PlaceTheCall, EmailAfterCall, 
## OneToOne, Status, CreationDate)

ExtractFromCRM('OnlineLeadFollowup', csvIndex='FollowupId')

In [19]:
## CRM.HistoricContact
## (HistoricContactId, Personid, InitialDate, FinalDate, ContactType, CandidateStatus, GeographicalArea, 
## Country, Province)

ExtractFromCRM('HistoricContact')

In [21]:
## Celda auxiliar de comprobación para cargar CSV y visualizar como DataFrame de pandas

## pd.read_csv('data/CRM.Person.csv.gz', sep = ';', compression = 'gzip', quotechar='|', quoting=csv.QUOTE_MINIMAL)

In [22]:
## Tablas provenientes de la Solicitud Online

## Se relacionan entre ellas por el identificador de persona de la solicitud online PersonIdOA, y pueden 
## relacionarse con el resto de CRM a través del atributo EmailAddress como se había acordado para que el 
## e-mail sirviese como nexo, y además, en los casos en que la solicitud online tenga informados en 
## OA.OnlineApplication los campos OpportunityId o EnrollmentId, se puede hacer un cruce directo con las 
## tablas CRM.Opportunity o CRM.Enrollment, respectivamente.

## Estas tablas son análogas a las de CRM, salvo que aquí no se excluyen registros inactivos, hay atributos 
## propios como TransferredToCRM, y hay dos tablas adicionales OA.OnlineApplication y OA.AppliedInstitution 
## con los datos de la solicitud online y escuelas aplicadas. También los recomendantes son diferentes, ya 
## que aquí provienen de cartas de referencia y se vinculan a la persona, anotándose en ellos su cargo y 
## ubicación, pudiendo ser de cualquier empresa, organismo o país, sin que estén catalogados, a diferencia 
## de en CRM que van vinculados a la admisión y se catalogan como recomendantes en CRM, generalmente personal
## del IE. Como a veces se queda el recomendante en blanco, se han incluido dos campos a nivel de persona 
## RecommendedOA y RecommendersOA para recoger en el primero si tiene recomendación o no, y en el segundo
## la lista de recomendantes rellenados junto con su posición y localización. 


In [5]:
## Procedimiento de extracción de datos de la Solicitud Online a un fichero CSV local

def ExtractFromOnlineApplication(tableName, **optionalParameters): 
    
    ## Motor SQLAlchemy para la base de datos SQL Server de la Solicitud Online
    
    oaEngine = sqlalchemy.create_engine(
         dbc.oa['dialect'] + '+' + dbc.oa['driver'] + '://' \
         + dbc.oa['username'] + ':' + dbc.oa['password'] + '@' \
         + dbc.oa['host'] + '/' + dbc.oa['database'] + dbc.oa['parameters'])
    
    # Ruta del fichero CSV, asumida en una subcarpeta "data" en la carpeta donde se ejecuta
    # y extensión .csv.gz porque se generará comprimido en formato GZIP
    
    csvFile = optionalParameters.get('csvFile', 'data/OA.' + tableName + '.csv.gz')
    
    # Parámetro opcional para indicar si requiere un procedimiento previo auxiliar "Aux"
    
    auxProcedure = optionalParameters.get('auxProcedure', None)
    
    # Parámetro opcional con el índice a aplicar al fichero CSV
    
    csvIndex = optionalParameters.get('csvIndex', None)
    if (csvIndex == None):
        csvIndex = tableName + 'Num'        
        
    # Recuperación por base de datos de tabla resultante de la transformación
    
    oaConnection = oaEngine.connect()
    sqlSentence = ''
    if (auxProcedure != None):
        sqlSentence += 'EXEC BI.' + auxProcedure + '\n'
    sqlSentence += "EXEC BI." + tableName
    if (auxProcedure != None):
        sqlSentence += '\nDROP TABLE Aux' + tableName + '\n'
    table = pd.io.sql.read_sql(sqlSentence, oaConnection)
    table.set_index(csvIndex, inplace = True)
    table.to_csv(csvFile, sep = ';', compression = 'gzip', quotechar='|', quoting=csv.QUOTE_MINIMAL, 
                 encoding = 'utf8')
    oaConnection.close()
      
        

In [6]:
## OA.AcademicTraining 
## (AcademicTrainingNum, AcademicTrainingForWhom, PersonIdOA, Institution, School, City, Province, Country, 
## InstitutionWeb, SchoolWeb, AcademicArea, AcademicSubarea, DegreeType, Title, SpecificLevelOfEducation, 
## SpecificEducationSystem, CurrentTraining, AverageGrade, MaxGrade, Language, Comment, TransferredToCRM, 
## StartingDate, EndingDate, ExpectedEndingDate)

ExtractFromOnlineApplication('AcademicTraining')



In [25]:
## OA.AdmissionTest 
## (TestNumber, PersonIdOA, TestName, IsAdmissionExam, IsAdmissionExamIEU, Score, Date, TransferredToCRM)

ExtractFromOnlineApplication('AdmissionTest', csvIndex='TestNumber')

In [26]:
## OA.AppliedInstitution 
##(AppliedInstitutionNum, PersonIdOA, GeographicalArea, Institution)

ExtractFromOnlineApplication('AppliedInstitution')

In [27]:
## OA.OnlineApplication 
## (OnlineApplicationNum, PersonIdOA, OpportunityId, ProgramId, OfferingId, EnrollmentId, ClassroomShift, 
## StartingDate, EndingDate, Progress, TransferredToCRM, TransferToCRMDate, Comment, TrackingAdwordsFin, 
## InterestOnDegrees, InterestType, WhyComeToIE, WhyChooseThisProgram, CompanyPayment, CompanyPaymentContactPerson, 
## CompanyPaymentContactPosition, CompanyPaymentOrganizationType, CompanyPaymentSector, CompanyPaymentWeb, 
## CompanyPaymentEmployees, CompanyPaymentAnnualSales, CompanyPaymentCIF, CompanyPaymentTelephone, 
## CompanyPaymentEmailAddress, PaymentPercent, PaymentValue, OnlinePayment, Price, LastUpdate)

ExtractFromOnlineApplication('OnlineApplication')

In [28]:
## OA.Person (PersonIdOA, Fullname, EmailAddress, Telephone, InterestTypeOA, HomeCountry, Alumni, MktCountry, 
## MktProvince, YearsOfExperience, RegistrationDate, SessionDate, LastUpdate, OnlinePayment, Paid, PaymentDate, 
## PaymentReference, AcceptedStatement, AcceptedStatementCorrectInfo, AlumniLOPD, LOPD, TransferredToCRM, 
## TransferInProgressToCRM, InterestOnDegrees, TransferStudent, ProgramOfInterest1, ProgramOfInterest2, 
## VisitedCountries, DailyOtherLanguageActivity, RecommendedOA, RecommendersOA)

ExtractFromOnlineApplication('Person', csvIndex='PersonIdOA')

In [29]:
## OA.ProfessionalExperience 
## (ProfessionalExperienceNum, PersonIdOA, Organization, OrganizationType, FamilyBusiness, OrganizationWeb, 
## City, Country, Sector, Employees, Area, Department, Title, ResponsibilityLevel, NumberOfDependants, 
## FixedAnnualSalary, VariableAnnualSalary, StartingDate, EndingDate, CurrentPosition, TravelTimeForWork,
## WorkingTimeForPeople, Functions, NumEmployees, SelfEmployed, TransferredToCRM, LastUpdate)

ExtractFromOnlineApplication('ProfessionalExperience')

In [30]:
## OA.LanguageCompetence 
## (LanguageCompetenceIdOA, PersonIdOA, Language, Level, TransferredToCRM, Lastupdate) 

ExtractFromOnlineApplication('LanguageCompetence', csvIndex='LanguageCompetenceIdOA')

In [31]:
## OA.NationalityRecord 
## (NationalityRecordIdOA, PersonIdOA, Nationality, DocumentType, Document, TransferredToCRM, LastUpdate)

ExtractFromOnlineApplication('NationalityRecord', csvIndex='NationalityRecordIdOA')

In [32]:
## Sincronización de carpeta local "data" con carpeta "tc-crm-solonline" del bucket S3 iedatalakeprocess
## Hace uso de AWSCLI (previamente instalado mediante la instrucción: sudo apt install awscli)

transformationFinalDate = str(datetime.datetime.now())
os.system('aws s3 sync data s3://iedatalakeprocess/tc-crm-solonline --metadata transformationprocess="' 
          + transformationProcess  + '",transformationversion="' + transformationVersion 
          + '",transformationinitialdate="'  + transformationInitialDate + '",transformationfinaldate="' 
          + transformationFinalDate + '",code="Transform-CRM-SolOnline.ipynb"');


In [33]:
## Borrado de la carpeta auxiliar "data" y todos los ficheros generados en ella (cuando proceso quede estable)
# shutil.rmtree('data')